In [ ]:
import math

class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._backward = lambda: None 
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f'Value({self.data})'
    
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad = out.grad
            other.grad = out.grad
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad = other.data * out.grad
            other.grad = self.data * out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        e_x = math.exp(self.data)
        out = Value(e_x, (self,), 'exp')
        def _backward():
            self.grad = e_x * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        self.grad = 1
        self._backward()
        # TODO: implement the backward pass


In [ ]:
a = Value(2.5)
b = Value(-1.5)
c = a + b

(1.0, {Value(-1.5), Value(2.5)}, '+')